<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import os
import pandas as pd
from zipfile import ZipFile
import matplotlib.pyplot as plt
import seaborn as sns
import platform

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
# Verificar si el archivo ya ha sido descargado
if os.access('./fake_news_dataset/Fake.csv', os.F_OK) is False:
    if os.access('Fake.csv', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/sevann-radhak/procesamiento_lenguaje_natural/main/clase_2/ejercicios/raw/Fake.csv -o Fake.csv
        else:
            !wget https://raw.githubusercontent.com/sevann-radhak/procesamiento_lenguaje_natural/main/clase_2/ejercicios/raw/Fake.csv -O Fake.csv
    # Crear la carpeta y mover el archivo
    os.makedirs('./fake_news_dataset', exist_ok=True)
    os.rename('Fake.csv', './fake_news_dataset/Fake.csv')
else:
    print("El archivo ya se encuentra descargado")

--2024-07-04 20:09:47--  https://raw.githubusercontent.com/sevann-radhak/procesamiento_lenguaje_natural/main/clase_2/ejercicios/raw/Fake.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62789876 (60M) [text/plain]
Saving to: ‘Fake.csv’

Fake.csv            100%[===================>]  59.88M   321MB/s    in 0.2s    

2024-07-04 20:09:49 (321 MB/s) - ‘Fake.csv’ saved [62789876/62789876]



In [ ]:
# # Posibles bandas
# os.listdir("./songs_dataset/")

In [3]:
df = pd.read_csv('./fake_news_dataset/Fake.csv')
print(df.head(5))

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  


In [4]:
# Combinar las columnas title y text en una sola columna llamada content
df['content'] = df.apply(lambda row: row['title'] + '.' + row['text'] if not row['title'].endswith('.') else row['title'] + ' ' + row['text'], axis=1)

# Eliminar las columnas date y subject
df.drop(columns=['date', 'subject', 'title', 'text'], inplace=True)

# Mostrar el resultado del DataFrame
print(df.head(5))

                                             content
0   Donald Trump Sends Out Embarrassing New Year’...
1   Drunk Bragging Trump Staffer Started Russian ...
2   Sheriff David Clarke Becomes An Internet Joke...
3   Trump Is So Obsessed He Even Has Obama’s Name...
4   Pope Francis Just Called Out Donald Trump Dur...


In [5]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 23481


### 1 - Preprocesamiento

In [6]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [7]:
# Demos un vistazo
sentence_tokens[:5]

[['donald',
  'trump',
  'sends',
  'out',
  'embarrassing',
  'new',
  'year’s',
  'eve',
  'message',
  'this',
  'is',
  'disturbing',
  'donald',
  'trump',
  'just',
  'couldn',
  't',
  'wish',
  'all',
  'americans',
  'a',
  'happy',
  'new',
  'year',
  'and',
  'leave',
  'it',
  'at',
  'that',
  'instead',
  'he',
  'had',
  'to',
  'give',
  'a',
  'shout',
  'out',
  'to',
  'his',
  'enemies',
  'haters',
  'and',
  'the',
  'very',
  'dishonest',
  'fake',
  'news',
  'media',
  'the',
  'former',
  'reality',
  'show',
  'star',
  'had',
  'just',
  'one',
  'job',
  'to',
  'do',
  'and',
  'he',
  'couldn',
  't',
  'do',
  'it',
  'as',
  'our',
  'country',
  'rapidly',
  'grows',
  'stronger',
  'and',
  'smarter',
  'i',
  'want',
  'to',
  'wish',
  'all',
  'of',
  'my',
  'friends',
  'supporters',
  'enemies',
  'haters',
  'and',
  'even',
  'the',
  'very',
  'dishonest',
  'fake',
  'news',
  'media',
  'a',
  'happy',
  'and',
  'healthy',
  'new',
  'yea

### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 23481


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 36929


### 3 - Entrenar embeddings

In [14]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=10,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 62406504.0
Loss after epoch 1: 20299648.0
Loss after epoch 2: 16783320.0
Loss after epoch 3: 16303752.0
Loss after epoch 4: 15747944.0
Loss after epoch 5: 2676560.0
Loss after epoch 6: 0.0
Loss after epoch 7: 0.0
Loss after epoch 8: 0.0
Loss after epoch 9: 0.0


(83167908, 105970810)

### 4 - Ensayar

In [15]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["darling"], topn=10)

[('neanderthal', 0.539341151714325),
 ('“tolerant”', 0.5232668519020081),
 ('partier', 0.5143733024597168),
 ('fronted', 0.5072934627532959),
 ('cocksucker', 0.5053715109825134),
 ('lonely', 0.5036835670471191),
 ('cackles', 0.4970766305923462),
 ('disowned', 0.49511852860450745),
 ('talker', 0.4945867955684662),
 ('stigmatized', 0.49107807874679565)]

In [16]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('iranian', 0.08956789970397949),
 ('ambulance', 0.051791638135910034),
 ('located', 0.0516662634909153),
 ('karl', 0.037720732390880585),
 ('highway', 0.03580015152692795),
 ('permanently', 0.031872596591711044),
 ('changes', 0.030781667679548264),
 ('bay', 0.030567022040486336),
 ('stocks', 0.027841215953230858),
 ('chai', 0.02754378318786621)]

In [21]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["donald"], topn=10)

[('trump', 0.5946163535118103),
 ('“president', 0.5357708930969238),
 ('impeaching', 0.5309218764305115),
 ('2016donald', 0.5273985862731934),
 ('pussygrabber', 0.5158053636550903),
 ('2017donald', 0.5131936073303223),
 ('hesitance', 0.5023653507232666),
 ('‘donald', 0.499002605676651),
 ('majorly', 0.4956330358982086),
 ('catapult', 0.4879090487957001)]

In [22]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["russia"], topn=10)

[('russian', 0.5876344442367554),
 ('russians', 0.5799474716186523),
 ('putin', 0.5575988292694092),
 ('vladamir', 0.5051841139793396),
 ('hacking”', 0.4892229437828064),
 ('specious', 0.48912477493286133),
 ('‘russian', 0.4795019030570984),
 ('putin’s', 0.47736117243766785),
 ('kremlin', 0.47502368688583374),
 ('crimean', 0.4733864963054657)]

In [19]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["africa"], topn=5)

[('europe', 0.49120330810546875),
 ('drc', 0.4744619131088257),
 ('asia', 0.445223867893219),
 ('hong', 0.44334346055984497),
 ('bangladesh', 0.43860769271850586)]

In [ ]:
# Ensayar con una palabra que no está en el vocabulario:
# w2v_model.wv.most_similar(negative=["diedaa"])

In [25]:
# el método `get_vector` permite obtener los vectores:
vector = w2v_model.wv.get_vector("africa")
print(vector)

[ 0.10041391  0.08814121 -0.02199912 -0.02916538  0.38246235 -0.74383444
 -0.2600821   0.14937487  0.29288372 -0.08521254 -0.2584028   0.0465727
 -0.26590323  0.3108723   0.06401    -0.13130566  0.6917744  -0.02813376
  0.02307568 -0.46996096 -0.20854345  0.11799757  0.5885583  -0.18231252
  0.17570122 -0.3532135  -0.6302653   0.68493766  0.08841715 -0.14960942
  0.6896987  -0.17433248 -0.61674917  0.6676924  -0.36949295 -0.04928316
 -0.29176494 -0.216082    0.11388417  0.310698    0.06601182 -0.09639623
 -0.22310545 -0.08280247 -0.22293213 -0.13741958  0.16017917 -0.04085771
 -0.03509033  0.47757646 -0.13524254  0.31402877 -0.37501806  0.22917065
  0.4326396   0.11868604  0.49102303  0.04716194 -0.22843918 -0.03001539
 -0.01559567 -0.14694881  0.15228945 -0.1833328  -0.23840824  0.16540451
  0.4291259   0.07598369 -0.71097744  0.42803752 -0.13357054  0.20414767
  0.0497214   0.14165984 -0.29144618  0.21166274  0.3618697  -0.7112854
  0.34895545  0.57840025 -0.3142842  -0.22079507  0.0

In [26]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector)

[('africa', 0.9999999403953552),
 ('europe', 0.49120330810546875),
 ('drc', 0.4744619131088257),
 ('asia', 0.445223867893219),
 ('hong', 0.44334346055984497),
 ('bangladesh', 0.43860769271850586),
 ('jungles', 0.4382091164588928),
 ('cambodia', 0.43656477332115173),
 ('burundi', 0.4342784583568573),
 ('peru', 0.4337144196033478)]

### 5 - Visualizar agrupación de vectores

In [38]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [39]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [40]:
# # Graficar los embedddings en 3D

# vecs, labels = reduce_dimensions(w2v_model,3)

# fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
# fig.update_traces(marker_size = 2)
# fig.show(renderer="colab") # esto para plotly en colab

In [41]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.